# 摘要生成

# encoder-decoder架构

In [1]:
import torch

from datasets import Dataset

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments

In [2]:
dataset = Dataset.load_from_disk("../../datas/nlpcc_2017")

In [5]:
dataset = dataset.train_test_split(100, seed=42)

In [6]:
dataset["train"][0]

{'title': '组图:黑河边防军人零下30℃户外训练,冰霜沾满眉毛和睫毛,防寒服上满是冰霜。',
 'content': '中国军网2014-12-1709:08:0412月16日,黑龙江省军区驻黑河某边防团机动步兵连官兵,冒着-30℃严寒气温进行体能训练,挑战极寒,锻造钢筋铁骨。该连素有“世界冠军的摇篮”之称,曾有5人24人次登上世界军事五项冠军的领奖台。(魏建顺摄)黑龙江省军区驻黑河某边防团机动步兵连官兵冒着-30℃严寒气温进行体能训练驻黑河某边防团机动步兵连官兵严寒中户外训练,防寒服上满是冰霜驻黑河某边防团机动步兵连官兵严寒中户外训练,防寒服上满是冰霜官兵睫毛上都被冻上了冰霜官兵们睫毛上都被冻上了冰霜驻黑河某边防团机动步兵连官兵严寒中进行户外体能训练驻黑河某边防团机动步兵连官兵严寒中进行户外体能训练驻黑河某边防团机动步兵连官兵严寒中进行户外体能训练'}

In [8]:
tokenizer = AutoTokenizer.from_pretrained("models/mengzi-T5")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [19]:
dataset["train"][0]

{'title': '组图:黑河边防军人零下30℃户外训练,冰霜沾满眉毛和睫毛,防寒服上满是冰霜。',
 'content': '中国军网2014-12-1709:08:0412月16日,黑龙江省军区驻黑河某边防团机动步兵连官兵,冒着-30℃严寒气温进行体能训练,挑战极寒,锻造钢筋铁骨。该连素有“世界冠军的摇篮”之称,曾有5人24人次登上世界军事五项冠军的领奖台。(魏建顺摄)黑龙江省军区驻黑河某边防团机动步兵连官兵冒着-30℃严寒气温进行体能训练驻黑河某边防团机动步兵连官兵严寒中户外训练,防寒服上满是冰霜驻黑河某边防团机动步兵连官兵严寒中户外训练,防寒服上满是冰霜官兵睫毛上都被冻上了冰霜官兵们睫毛上都被冻上了冰霜驻黑河某边防团机动步兵连官兵严寒中进行户外体能训练驻黑河某边防团机动步兵连官兵严寒中进行户外体能训练驻黑河某边防团机动步兵连官兵严寒中进行户外体能训练'}

In [11]:
tokenizer.convert_ids_to_tokens(7)

'▁'

In [14]:
def process_func(examples):
    prefix = "摘要生成: \n"
    contents = [prefix + content for content in examples["content"]]
    inputs = tokenizer(contents, max_length=384, truncation=True, padding=True)
    labels = tokenizer(text_target=examples["title"], max_length=64, truncation=True)
    inputs["labels"] = labels["input_ids"]
    return inputs

In [15]:
tokenized_data = dataset.map(process_func, batched=True, remove_columns=dataset["train"].column_names)

Map:   0%|          | 0/4900 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
print(tokenized_data["train"][0])
print(tokenizer.decode(tokenized_data["train"][0]["input_ids"]))
print(tokenizer.decode(tokenized_data["train"][0]["labels"]))

{'input_ids': [7, 17965, 5353, 13, 5893, 349, 295, 2927, 13049, 114, 1238, 6473, 13, 5844, 13, 6849, 364, 50, 1095, 70, 3, 15059, 12850, 3168, 409, 542, 717, 505, 1139, 877, 20399, 17187, 375, 14061, 3, 18694, 11081, 10788, 722, 24698, 6472, 103, 16773, 1332, 3, 2197, 1164, 1342, 3, 26664, 12966, 789, 1195, 4, 206, 375, 18665, 31, 304, 2808, 5, 24456, 41, 12562, 3, 17222, 108, 21, 1367, 14434, 8768, 304, 2898, 270, 1221, 2808, 5, 1324, 1065, 498, 4, 22, 2962, 606, 1214, 5588, 25, 15059, 12850, 3168, 409, 542, 717, 505, 1139, 877, 20399, 17187, 375, 14061, 18694, 11081, 10788, 722, 24698, 6472, 103, 16773, 1332, 3168, 409, 542, 717, 505, 1139, 877, 20399, 17187, 375, 14061, 24698, 16, 5968, 1332, 3, 1139, 1342, 1163, 23, 586, 11, 1301, 3883, 3168, 409, 542, 717, 505, 1139, 877, 20399, 17187, 375, 14061, 24698, 16, 5968, 1332, 3, 1139, 1342, 1163, 23, 586, 11, 1301, 3883, 14061, 13424, 23, 5599, 4476, 2037, 1301, 3883, 14061, 324, 13424, 23, 5599, 4476, 2037, 1301, 3883, 3168, 409, 542, 

In [20]:
model = AutoModelForSeq2SeqLM.from_pretrained("models/mengzi-T5")

In [26]:
print(tokenized_data["train"][11])

{'input_ids': [7, 17965, 5353, 13, 7, 1947, 6806, 3965, 2930, 2313, 138, 40, 3, 1947, 9100, 403, 4196, 3, 1228, 1826, 1727, 3294, 3304, 3028, 504, 13446, 18, 173, 1258, 131, 3, 151, 1918, 13101, 108, 131, 3, 10026, 3670, 24298, 204, 18, 2263, 99, 131, 3, 242, 151, 1918, 8283, 138, 131, 4, 11695, 3, 24462, 391, 159, 13446, 350, 3, 941, 3294, 3304, 3028, 504, 13446, 18, 138, 1704, 131, 3, 19, 1918, 3562, 3, 10026, 3670, 24298, 204, 18, 108, 419, 131, 3, 151, 1918, 13101, 165, 131, 4, 1453, 3, 678, 3594, 2188, 131, 8952, 23624, 131, 3, 151, 1918, 476, 27171, 138, 131, 3, 219, 880, 131, 547, 2262, 768, 218, 6041, 21, 3, 20392, 2133, 151, 1918, 65, 98, 4739, 100, 11270, 65, 98, 2392, 7456, 4, 6784, 131, 547, 1084, 138, 1863, 21, 3, 20392, 2133, 2303, 4112, 219, 14476, 151, 1918, 65, 16987, 219, 21, 13047, 24, 391, 15056, 13446, 2927, 24, 391, 15056, 13446, 3728, 6806, 3965, 9709, 24478, 924, 2853, 1299, 1947, 2416, 9100, 391, 15056, 13446, 4, 1113, 345, 3, 678, 391, 15056, 10026, 3670, 2429

In [ ]:
import numpy as np
from rouge_chinese import Rouge

rouge = Rouge()

def compute_metrics(pred):
    predictions, labels = pred
    predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    score = rouge.get_scores(predictions, labels, avg=True)
    return {
        "rouge-1": score["rouge-1"]["f"],
        "rouge-2": score["rouge-2"]["f"],
        "rouge-l": score["rouge-l"]["f"] # rouge 有 1-5 and l, 有r, p, f
    }

In [28]:
args = Seq2SeqTrainingArguments(
    output_dir="trained/model_for_summary_T5",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    per_device_eval_batch_size=16,
    optim="adafactor",
    learning_rate=1e-5,
    weight_decay=0.01,
    logging_steps=200,
    metric_for_best_model="rouge-l",
    predict_with_generate=True
)

In [29]:
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    compute_metrics=compute_metrics,
    data_collator=DataCollatorForSeq2Seq(tokenizer)
)

In [32]:
trainer.train()

Step,Training Loss


KeyboardInterrupt: 

In [ ]:
from transformers import T5ForConditionalGeneration

transformers.models.t5.modeling_t5.T5ForConditionalGeneration

In [33]:
from transformers import pipeline

pipe = pipeline("text2text-generation", tokenizer=tokenizer, model=model)

Device set to use cuda:0


In [36]:
pipe("摘要生成: \n" + tokenizer.decode(tokenized_data["test"][-1]["input_ids"], skip_special_tokens=True))

[{'generated_text': '中船重工拟对所持动力业务进行对外投资,涉及公司股票停牌。'}]

In [37]:
tokenizer.decode(tokenized_data["test"][-1]["labels"])

'中国重工拟以持有的动力相关资产进行对外投资,参与中船重工拟打造的动力业务平台公司,将继续停牌。</s>'

# prefix-model

In [32]:
import torch

from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments

In [2]:
dataset = Dataset.load_from_disk("../../datas/nlpcc_2017")

In [3]:
dataset = dataset.train_test_split(100, seed=42)

In [ ]:
'''
修改tokenization_glm.py源码
class GLMChineseTokenizer(PreTrainedTokenizer, GLMTokenizerMixin):
    vocab_files_names = {"vocab_file": "cog-pretrain.model"}
    truncation_side: str = "left"

    def __init__(self, vocab_file, **kwargs):
        self.vocab_file = vocab_file
        self.sp_model = spm.SentencePieceProcessor()
        self.sp_model.Load(vocab_file)
        super().__init__(**kwargs)

因为调用顺序PreTrainedTokenizer.__init__->_add_tokens->self.get_vocab().copy()
但是self.get_vocab()在GLMChineseTokenizer定义如下
    @property
    def vocab_size(self):
        return len(self.sp_model)

    def get_vocab(self):
        vocab = {self.convert_ids_to_tokens(i): i for i in range(self.vocab_size)}
        vocab.update(self.added_tokens_encoder)
        return vocab

因此还没有加载self.sp_model.Load(vocab_file)就调用，报错，将父类初始化放在加载后就行
'''
tokenizer = AutoTokenizer.from_pretrained("models/glm-large", trust_remote_code=True)

In [14]:
dataset["train"][0]

{'title': '组图:黑河边防军人零下30℃户外训练,冰霜沾满眉毛和睫毛,防寒服上满是冰霜。',
 'content': '中国军网2014-12-1709:08:0412月16日,黑龙江省军区驻黑河某边防团机动步兵连官兵,冒着-30℃严寒气温进行体能训练,挑战极寒,锻造钢筋铁骨。该连素有“世界冠军的摇篮”之称,曾有5人24人次登上世界军事五项冠军的领奖台。(魏建顺摄)黑龙江省军区驻黑河某边防团机动步兵连官兵冒着-30℃严寒气温进行体能训练驻黑河某边防团机动步兵连官兵严寒中户外训练,防寒服上满是冰霜驻黑河某边防团机动步兵连官兵严寒中户外训练,防寒服上满是冰霜官兵睫毛上都被冻上了冰霜官兵们睫毛上都被冻上了冰霜驻黑河某边防团机动步兵连官兵严寒中进行户外体能训练驻黑河某边防团机动步兵连官兵严寒中进行户外体能训练驻黑河某边防团机动步兵连官兵严寒中进行户外体能训练'}

In [57]:
def process_func(examples):
    prefix = "摘要生成: \n"
    contents = [prefix + content + tokenizer.mask_token for content in examples["content"]]
    inputs = tokenizer(contents, max_length=384, truncation=True, padding="max_length", return_tensors="pt") # [CLS] contens[MASK] <|endoftext|>
    inputs = tokenizer.build_inputs_for_generation(inputs, targets=examples["title"], max_gen_length=64, padding=True) # [CLS] contents[MASK] <|endoftext|> <|startofpiece|> labels
    return inputs

In [58]:
tokenized_data = dataset.map(process_func, batched=True, remove_columns=dataset["train"].column_names)

Map:   0%|          | 0/4900 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [59]:
print(tokenized_data["train"][0]["position_ids"])

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221

In [60]:
model = AutoModelForSeq2SeqLM.from_pretrained("models/glm-large", trust_remote_code=True)

In [61]:
args = Seq2SeqTrainingArguments(
    output_dir="trained/model_for_summary_glm",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    per_device_eval_batch_size=16,
    optim="adafactor",
    learning_rate=1e-5,
    weight_decay=0.01,
    logging_steps=200,
    num_train_epochs=1
)

In [62]:
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=tokenized_data["train"],
    tokenizer=tokenizer,
)

C:\Users\17521\AppData\Local\Temp\ipykernel_17528\1714688615.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [63]:
trainer.train()

Step,Training Loss


KeyboardInterrupt: 

In [64]:
import torch

model = model.eval()

def predict_test():
    predict = []
    with torch.inference_mode():
        for d in dataset["test"]:
            inputs = tokenizer("摘要生成: \n" + d["content"] + tokenizer.mask_token, return_tensors="pt")
            inputs = tokenizer.build_inputs_for_generation(inputs, max_gen_length=64)
            inputs = inputs.to("cuda")
            output = model.generate(**inputs, max_new_tokens=64, eos_token_id=tokenizer.eop_token_id, do_sample=True)
            predict.append(tokenizer.decode(output[0].tolist()).split("<|startofpiece|>")[1].replace("<|endofpiece|>", "").strip())
            print("curID:", len(predict))
    return predict

In [66]:
result = predict_test()

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50007 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50007 for open-end generation.


curID: 1


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50007 for open-end generation.


curID: 2


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50007 for open-end generation.


...

curID: 97


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50007 for open-end generation.


curID: 98


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50007 for open-end generation.


curID: 99
curID: 100


In [72]:
result

['环球时报:美国担心黑客获取政府公务人员信息,拟将他们列为攻击对象。',
 '安徽:宿松招商局下属2副局长吸毒品被“通报”,分管副局长被党内警告。 李同保被开除党籍、行政撤职、开除其股级、停发其工资。',
 '黑龙江每天平均接到400个"伪基站"投诉电话',
 '本周尾号限行规则: 北京周末晚高峰,北京交通压力明显,建议尽量乘坐公共交通工具。',
 '哈市警方:与儿媳闹不和,离家出走多年,儿子找妻子无果竟厮打在一起。',
 '在苏州举行初三联考,中考命题突然泄露,校长被停职:试卷泄露给了自己的亲戚,并散播出去;',
 '河北省承德2名处级干部因严重违纪被开除党籍,其涉嫌犯罪问题及线索均移送司法机关依法处理。',
 '霍比特人陈佩斯微博晒中文签名自黑:我是佩克;',
 '摘录: 中新社 叙利亚:从“真主党”手中获小麦占到伊拉克年小麦消费量1/3',
 '昨日北京发布大风蓝色预警 天津渤海、渤海海峡及沿海、大连有9级大风,天津机场发布大风蓝色预警',
 '上海浦东开发开放25年 上海人口老龄化严重 昂山素季:中国25年来取得巨大变化',
 '湖南凤凰第一寿星已122岁!自称摇了一辈子蒲扇,身体健康,可活到150岁。',
 '',
 '腰缠万贯”',
 '华科街头摆摊招聘女友,自称是真心想找个女友:',
 '湖北省荆州市发布霾黄色预警:目前我市部分地区已经出现能见度小于5公里的霾 未来仍将持续,请注意防范。',
 '晒与姐姐合影网友偶遇:娜扎疑扮嫩撞脸黄磊娇妻',
 '莱州已建光伏发电板200余块 用于居民照明 每天发8千度电',
 '实习生 李倩;',
 '5月份郑州市商品住宅销售均价达10243元/平方米; 郑州市商品住宅均价首次突破万元;',
 '在过去的一年时间内,英特尔斥资超过170亿美元收购了一家芯片企业,但Altera表现并不理想,至今尚无任何进展。',
 '报道,近日家住舒城,安徽舒城被妻子算计,婚生子被母亲带走,一怒之下提起离婚的男子诉称,被妻子的同事在结婚两年后偷走2万元。 [1]',
 '徐新民曾在瑞昌任职,其所在的桂林街道办事处在瑞昌市经济工作会议上,曾被称为“瑞昌第二街道”。',
 '湖南衡阳:一男疑雇陌生保安多次拨打医院电话称炸弹要爆炸,编造虚假恐怖信息获刑一年',
 '汉中市住建局一工作人员称,沟通过后,业主才抱他腿,“当时他就一直对我微笑

In [73]:
dataset["test"]["title"]

['“伊斯兰国”网上公布暗杀名单,锁定1400名官员,大部分为美国人,详列姓名、手机等资料。',
 '安庆宿松两副局长吸毒受处分 所在单位被通报批评',
 '黑龙江:报道揭露黑龙江”伪基站“盛行,每天向百万手机用户发放垃圾短信。',
 '明日北京将启动新一轮车牌限行尾号轮换,周一至周五分别为2和7、3和8、4和9、5和0、1和6',
 '男子出轨多日不回家,向妻子提出离婚,被找到后遭母亲当街痛打5分钟,辅警赶来后将三人劝和。',
 '苏州一初中副校长涉嫌泄露联考题目给亲戚,导致题目被传到网上,副校长停职被纪委调查;联考成绩直接影响考生能否进重点高中。',
 '承德2名处级干部严重违纪被开除党籍、开除公职,涉嫌犯罪问题及线索已移送司法机关处理。',
 '精灵王李·佩斯开通微博,贴照片书汉字向网友问好;汉字横平竖直被网友怀疑“用尺子量着写的” ',
 '分析报告称,IS把大量伊拉克小麦转运到叙利亚磨成面粉,通过“过境土耳其”向其他地区出售,每年获利约两亿美元',
 '天津市发布大风黄色预警:预计今日夜间到明天白天渤海海面,将有西北风9级,阵风10-11级。请有关单位和人员作好防...',
 '上海市委书记韩正傍晚会见昂山素季,昂山素季称赞上海取得巨大经济成就,并表示敬佩中共反腐决心和力度。',
 '湖南第一寿星高龄122岁,头发半黑没有老年斑;生育13个孩子,42年前老伴去世后开始独自生活',
 '杭州拱墅区大关街道一科级干部被举报强奸女员工,并与其保持情人关系2年多;目前纪委已介入调查',
 '深圳:女子腰藏16斤黄金闯关被查,因走路时腰部僵硬露陷,共藏8块金条,价值200万元',
 ' 23岁男子高校内摆摊招聘女友,自定一个月“试用期”,称感觉良好可提前转正,摆摊1个多小时无人应征',
 '荆州市发布霾黄色预警:目前我市部分地区已经出现能见度小于5公里的霾,未来仍将持续,请注意防范。...',
 '[拌饭]成龙洗发水广告遭恶搞,"DUANG"成网络热词;朱孝天自曝恋情:就这么定了;古力娜扎晒家人合影, 姐姐撞脸黄磊娇妻',
 '烟台:农民在屋顶建光伏发电系统,靠卖电月入6000余元,感慨称“太阳光也能变成钱”',
 '宝鸡:七八岁的姐弟俩怄气离家出走,失踪4天后河中现姐姐尸体,弟弟仍不知去向。',
 '5月郑州商品住宅均价达10243元/平方米,首次突破万元;分析称